In [24]:
import numpy as np
import pandas as pd
from keras.utils import to_categorical

data_train = pd.read_csv("/Users/rahmi/Downloads/fashion-mnist_train.csv")
data_train.head(5)

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
data_test = pd.read_csv("/Users/rahmi/Downloads/fashion-mnist_test.csv")

In [26]:
img_rows,img_cols = 28,28
input_shape = (img_rows,img_cols,1)

In [48]:
img_rows

28

In [27]:
X = np.array(data_train.iloc[:,1:])
y = to_categorical(np.array(data_train.iloc[:,0]))

In [28]:
#Test data
X_test = np.array(data_test.iloc[:, 1:])
y_test = to_categorical(np.array(data_test.iloc[:, 0]))

In [29]:
from sklearn.model_selection import train_test_split

X_train,X_val,y_train,y_val = train_test_split(X,y,test_size = 0.2,random_state = 13)

In [30]:
X_train = X_train.reshape(X_train.shape[0],img_rows,img_cols,1)
X_val = X_val.reshape(X_val.shape[0],img_rows,img_cols,1)
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)

In [31]:
X_train.shape

(48000, 28, 28, 1)

In [32]:
X_train = X_train.astype('float32')
X_val = X_val.astype('float32')
X_test = X_test.astype('float32')

In [33]:
X_train /= 255
X_val /= 255
X_test /= 255

In [34]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization

In [35]:
batch_size = 256
num_classes = 10
epochs = 2

In [36]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 kernel_initializer='he_normal',
                 input_shape=input_shape))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Dropout(0.4))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])

In [37]:
history = model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_val, y_val))
score = model.evaluate(X_test, y_test, verbose=0)

Train on 48000 samples, validate on 12000 samples
Epoch 1/2
48000/48000 [==============================] - 87s 2ms/step - loss: 0.8217 - acc: 0.6878 - val_loss: 0.5312 - val_acc: 0.8057
Epoch 2/2
48000/48000 [==============================] - 85s 2ms/step - loss: 0.5249 - acc: 0.8054 - val_loss: 0.4306 - val_acc: 0.8453


In [38]:
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.42085162415504457
Test accuracy: 0.8493


In [44]:
#get the predictions for the test data
predicted_classes = model.predict_classes(X_test)

#get the indices to be plotted
y_true = data_test.iloc[:, 0]

In [45]:
#get the indices to be plotted
#y_true = data_test.iloc[:, 0]
correct = np.nonzero(predicted_classes==y_test)[0]
incorrect = np.nonzero(predicted_classes!=y_test)[0]

/Users/rahmi/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/rahmi/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: elementwise != comparison failed; this will raise an error in the future.
  after removing the cwd from sys.path.


In [46]:
from sklearn.metrics import classification_report
target_names = ["Class {}".format(i) for i in range(num_classes)]
print(classification_report(y_true, predicted_classes, target_names=target_names))

             precision    recall  f1-score   support

    Class 0       0.78      0.81      0.79      1000
    Class 1       0.98      0.97      0.98      1000
    Class 2       0.82      0.65      0.72      1000
    Class 3       0.86      0.92      0.89      1000
    Class 4       0.68      0.85      0.76      1000
    Class 5       0.97      0.94      0.96      1000
    Class 6       0.60      0.54      0.57      1000
    Class 7       0.89      0.95      0.92      1000
    Class 8       0.99      0.94      0.96      1000
    Class 9       0.95      0.93      0.94      1000

avg / total       0.85      0.85      0.85     10000

